In [ ]:
# ============================================
__title__ = 'Modelling guided waves'
__author__ = "Abdalraheem A. Ijjeh"
__maintainer__ = "Abdalraheem A. Ijjeh"
__email__ = "aijjeh@imp.gda.pl"
# ============================================

In [ ]:
import os
import numpy as np
import json
import csv
import tensorflow as tf
import neptune.new as neptune
import matplotlib.pyplot as plt
import PIL
import re
import neptune.new as neptune

from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
from PIL import ImageOps
from tensorflow.keras.models import Model
from keras import backend as K
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score,  mean_squared_error
from decouple import config
from dotenv import load_dotenv
load_dotenv()

# Link to Neptune

In [ ]:
# access_token = os.getenv('NEPTUNE_API_TOKEN')
# run = neptune.init_run(project='abdalraheem.ijjeh/Guided-waves-modelling',
#                        api_token=access_token,
#                        tags=['DL HR_FFT2D_guided_waves_modelling', 'Fourier_domain']
#                        )

# Hyperparameters

In [ ]:
# run["Signal_based"] = "AE"
params = {'batches': 1,
          'num_filters': 24,
          'kernel_size': 3,
          'shape': (32, 32),
          'epochs': 10000,
          'dropout': 0.2,
          'levels': 4,
          'learning_rate': 0.00014329,
          'patience_epochs': 100,
          'val_split': 0.08,
          'hidden_layer': 3,
          'decay:steps': 100000,
          'decay_rate': 0.96,
          'time_stamps': 32
          }

# run["model/parameters"] = params

# Run on GPU

In [ ]:
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = '2'
# 
# physical_devices = tf.config.list_physical_devices('GPU')
# for device in physical_devices:
#     tf.config.experimental.set_memory_growth(device, True)

# Save the hyperparameters

In [ ]:
# env_path = '/home/aijjeh/Desktop/Phd_Projects/Modeling_guided_waves/'
# os.chdir(env_path)
# json = json.dumps(params)
# f = open('hyper_par_GWM_mse.json', 'w')
# f.write(json)
# f.close()

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
input_dir = "/aijjeh_odroid_sensors/aidd/data/raw/num/wavefield_dataset_undelam_bottom_out/1_output"
target_dir_ = "/aijjeh_odroid_sensors/aidd/data/raw/num/wavefield_dataset2_bottom_out"
my_list = os.listdir(input_dir)
my_list.sort(key=natural_keys)

In [ ]:
input_img_paths_total = []
target_img_paths_total = []

In [ ]:
for k in range(475):
    input_img_paths = sorted( 
        [
            os.path.join(input_dir, fname) 
            for fname in os.listdir(input_dir)    
            if fname.endswith(".png")
        ] 
    )
    input_img_paths.sort(key=natural_keys)
    input_img_paths_total.append(input_img_paths)

In [ ]:
Image(input_img_paths_total[0][80])

In [ ]:
print(len(input_img_paths_total))

In [ ]:
for i in range(475):
    target_dir = target_dir_ +'/%d_output' % (i+1) #str(my_list[i])
    target_img_paths = sorted( 
        [
            os.path.join(target_dir, fname) 
            for fname in os.listdir(target_dir)    
            if fname.endswith(".png")
        ] 
    )
    target_img_paths.sort(key=natural_keys)
    target_img_paths_total.append(target_img_paths)

In [ ]:
Image(target_img_paths_total[0][80])

In [ ]:
print(len(target_img_paths_total))

In [ ]:
os.chdir('/home/aijjeh/Desktop/Phd_Projects/Sequence_prediction/Full_wavefield_frames_time_series_project/Datasets/label_set')
file_frame = np.load('frames_initial.npy')
file_frame = file_frame[:380]
print(file_frame[0])

In [ ]:
os.chdir('/home/aijjeh/Desktop/Phd_Projects/Modeling_guided_waves/dataset')
X_train_1 = np.load('LR_GT_del.npy')
arr = X_train_1.reshape((475, 1, 5))
arr = np.repeat(arr, 32, axis=1)
print(arr.shape)
arr = arr.reshape((475, 32,1,5))
arr = np.repeat(arr, 32, axis=2)
print(arr.shape)
arr = arr.reshape((475,1,32,32,5))
arr = np.repeat(arr, 32, axis=1)

print(arr.shape)
arr = arr[:380]

print(arr.shape)

In [ ]:
class Full_wavefield_frames(tf.keras.utils.Sequence):
    def __init__(self, batch_size,
                 img_size, input_imgs_paths_total,
                 target_imgs_paths_total,
                 input_coords,
                 time_stamps):
        
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths_total = input_imgs_paths_total
        self.target_img_paths = target_imgs_paths_total
        self.input_coord = input_coords
        self.time_stamps = time_stamps

    def __len__(self):
        return len(self.input_img_paths_total) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        z = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths_total[z:z + self.batch_size]
        batch_target_img_paths = self.target_img_paths[z:z + self.batch_size]
        batch_input_coords = self.input_coord[z:z + self.batch_size]

        x1 = np.zeros((self.batch_size,) + (self.time_stamps,) + self.img_size + (1,), dtype="float16")  #
        x2 = np.zeros((self.batch_size,) + (self.time_stamps,) + self.img_size + (5,), dtype="float16")  #

        x_in = np.zeros((self.batch_size,) + (self.time_stamps,) + self.img_size + (6,), dtype="float16")  #

        for batch_num in range(self.batch_size):
            batch_input_img_paths = batch_input_img_paths[batch_num][file_frame[z] - 8: file_frame[z] + 24]
            batch_input_coords = batch_input_coords[batch_num][:]  # file_frame[z] - 8: file_frame[z] + 8
            count = 0
            for j, path in enumerate(batch_input_img_paths):
                img = load_img(path, target_size=self.img_size, color_mode="grayscale")
                img = np.expand_dims(img, 2)
                img = img / 255.0

                x1[batch_num][j] = img

                coords = arr[batch_num][count]
                x2[batch_num][j] = coords
                count += 1
            x_in = np.concatenate([x1, x2], axis=-1)

        y = np.zeros((self.batch_size,) + (self.time_stamps,) + self.img_size + (1,), dtype="float16")  #

        for batch_num in range(self.batch_size):
            batch_target_img_paths = batch_target_img_paths[batch_num][file_frame[z] - 8:file_frame[z] + 24]
            for j, path in enumerate(batch_target_img_paths):
                img = load_img(path, target_size=self.img_size, color_mode="grayscale")
                img = np.expand_dims(img, 2)
                img = img / 255.0
                y[batch_num][j] = img

        return x_in, y

# Split our img paths into a training and a validation set

In [ ]:
train_input_img_paths = input_img_paths_total[:304]
# print(len(train_input_img_paths))
train_target_img_paths = target_img_paths_total[:304]
# print(len(train_target_img_paths))
val_input_img_paths = input_img_paths_total[304:380]
# print(len(val_input_img_paths))
val_target_img_paths = target_img_paths_total[304:380]


# Instantiate data Sequences for each split
train_gen = Full_wavefield_frames(params['batches'],
                                  params['shape'],
                                  train_input_img_paths, 
                                  train_target_img_paths,
                                  arr[:304],
                                  params['time_stamps']) 

val_gen = Full_wavefield_frames(params['batches'],
                                params['shape'], 
                                val_input_img_paths, 
                                val_target_img_paths,
                                arr[304:],
                                params['time_stamps'])
print('0 axis',len(train_gen[0]))
print('1 axis',len(train_gen[0][0]))
print('2 axis',len(train_gen[0][0][0]))
print('3 axis',len(train_gen[0][0][0][0]))
print('4 axis',len(train_gen[0][0][0][0][0]))
print('5 axis',len(train_gen[0][0][0][0][0][0]))

In [ ]:
print(type(train_gen))

In [ ]:
x = []
y = []
for i in range(304):
    x.append(train_gen[i][0])
    y.append(train_gen[i][1])


In [ ]:
def get_time_distributed(time_input, n_filters):
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(filters=n_filters,
                               kernel_size=params['kernel_size'],
                               strides=1, 
                               padding='same',
                               activation='relu'))(time_input)
    return x

In [ ]:
def normalize_batch(bn_input):
    return tf.keras.layers.BatchNormalization()(bn_input)

In [ ]:
def custom_loss_FFT2D(y_true, y_pred):
    y_true_k = y_true
    y_pred_k = y_pred
    
    fft2d_true = tf.transpose(y_pred_k,[])
    fft2d_true = tf.signal.fft(tf.cast(y_true_k, dtype=tf.complex64))
    fft2d_pred = tf.signal.fft(tf.cast(y_pred_k, dtype=tf.complex64))
    
    N = tf.size(fft2d_true)
    N = tf.cast(N, dtype=tf.float32)

    fft2d_true = tf.signal.fftshift(fft2d_true, axes=(-1))
    fft2d_pred = tf.signal.fftshift(fft2d_pred, axes=(-1))

    fft2d_true = tf.cast(fft2d_true, dtype=tf.float32)
    fft2d_pred = tf.cast(fft2d_pred, dtype=tf.float32)

    fft2d_pred = tf.divide(fft2d_pred, N)
    fft2d_true = tf.divide(fft2d_true, N)

    MSE_Fourier_domain = tf.losses.MSE(abs(fft2d_true), abs(fft2d_pred))
    MSE_Spatial = tf.losses.MSE(y_true, y_pred)

    return MSE_Fourier_domain 

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    params['learning_rate'],
    decay_steps=10000,
    decay_rate=0.96,
    staircase=False)

In [ ]:
def get_model():
    
    inputs = tf.keras.layers.Input(shape=((params['time_stamps'],)+ (params['shape']) + (6,)))   
    print(inputs.shape)
    ####################################################################################################################
    encoder = get_time_distributed(inputs, params['num_filters'])
   
    skip_tensor = []
    for i in range(params['levels']):
        encoder = get_time_distributed(encoder, params['num_filters'])  
        encoder = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool2D((2, 2), strides=(2, 2)))(encoder)
        skip_tensor.append(encoder)
    print(encoder)
    ####################################################################################################################
    
    bottleneck = get_time_distributed(encoder, params['num_filters']*5)
    bottleneck = normalize_batch(bottleneck)
    bottleneck = get_time_distributed(bottleneck, params['num_filters']*5)
    bottleneck = normalize_batch(bottleneck)
    
    decoder = bottleneck
   # ####################################################################################################################
    
    for j in (range(1, params['levels'] + 1)):
        decoder = tf.keras.layers.concatenate((decoder, skip_tensor[-j]))
        decoder = tf.keras.layers.TimeDistributed(tf.keras.layers.UpSampling2D((2, 2)))(decoder)        
        decoder = get_time_distributed(decoder, params['num_filters'])
            
    ####################################################################################################################
    lstm_layer = tf.keras.layers.ConvLSTM2D(24, (1, 1), padding='same', return_sequences=True)(decoder)
    ####################################################################################################################
    # Output layer
   
    #################################################################################################################### 
    model_ = Model(inputs=inputs, outputs=lstm_layer)
    ####################################################################################################################
    model_.compile(tf.keras.optimizers.Adam(lr_schedule),
                  loss='mse',
                  metrics=[tf.keras.metrics.CosineSimilarity()],
                  run_eagerly=False)
    return model_

In [ ]:
tf.keras.backend.clear_session()
# Build model
model = get_model()
model.summary()

In [ ]:
checkpoint_filepath = env_path + 'temp/checkpoint/HR_frame_modelling_%s_%d_lr_filters_%d_levels_%d_batches_%d_epochs_%d_dropout_%s_val_split_%s_hidden_%d.h5' % (
    params['samples'],
    params['learning_rate'],
    params['num_filters'],
    params['levels'],
    params['batches'],
    params['epochs'],
    params['dropout'],
    params['val_split'],
    params['hidden_layer'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=params['patience_epochs'],
                                              mode='min'),
             tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                monitor='val_loss',
                                                save_best_only=True)]


class MonitoringCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        for metric_name, metric_value in logs.items():
            run[metric_name].log(metric_value)


In [ ]:
model.fit(train_gen,
          batch_size=params['batches'],
          validation_data=val_gen,
          epochs=params['epochs'],
          callbacks=[callbacks, MonitoringCallback()]) # 
